<a href="https://colab.research.google.com/github/le-pigeon/CEG5103/blob/main/MQTT_Stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependencies

In [7]:
!pip install paho-mqtt


Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.


Copied from https://nothans.com/thingspeak-tutorials/update-a-thingspeak-channel-using-mqtt-on-a-raspberry-pi

For test. Correct implementation to thingspeak below.

In [ ]:
# Publich to a ThingSpeak Channel Using MQTT
#
# This is an example of publishing to multiple fields simultaneously.
# Connections over standard TCP, websocket or SSL are possible by setting
# the parameters below.
#
# CPU and RAM usage is collected every 20 seconds and published to a
# ThingSpeak channel using an MQTT Publish
#
# This example requires the Paho MQTT client package which
# is available at: http://eclipse.org/paho/clients/python
import paho.mqtt.publish as publish
import string

# The ThingSpeak Channel ID.
# Replace <YOUR-CHANNEL-ID> with your channel ID.
channel_ID = "2879119"

# The hostname of the ThingSpeak MQTT broker.
mqtt_host = "mqtt3.thingspeak.com"

# Your MQTT credentials for the device
mqtt_client_ID = "CDAxARguGDMsFxYCDDcmOjE"
mqtt_username  = "CDAxARguGDMsFxYCDDcmOjE"
mqtt_password  = "IUZXvw0Wpn17v6l+Rt9tPIMT"

t_transport = "websockets"
t_port = 80

# Create the topic string.
topic = "channels/" + channel_ID + "/publish"

while (True):

    # get the system performance data over 20 seconds.
    cpu_percent = "5"
    ram_percent = "20"

    # build the payload string.
    payload = "field1=" + str(cpu_percent) + "&field2=" + str(ram_percent)

    # attempt to publish this data to the topic.
    try:
        print ("Writing Payload = ", payload," to host: ", mqtt_host, " clientID= ", mqtt_client_ID, " User ", mqtt_username, " PWD ", mqtt_password)
        publish.single(topic, payload, hostname=mqtt_host, transport=t_transport, port=t_port, client_id=mqtt_client_ID, auth={'username':mqtt_username,'password':mqtt_password})
    except Exception as e:
        print (e)

Run this one below to publish from CSV file.

In [ ]:
import pandas as pd
import requests
import json
import time
import io
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish
import string
from datetime import datetime
import pytz   # For date and time to SG time


# Publich to a ThingSpeak Channel Using MQTT
#
# This is an example of publishing to multiple fields simultaneously.
# Connections over standard TCP, websocket or SSL are possible by setting
# the parameters below.
#
# CPU and RAM usage is collected every 20 seconds and published to a
# ThingSpeak channel using an MQTT Publish
#
# This example requires the Paho MQTT client package which
# is available at: http://eclipse.org/paho/clients/python

#############################
# Please change the details below or you'll just be sending to my test channel
#############################

# The ThingSpeak Channel ID.
# Replace <YOUR-CHANNEL-ID> with your channel ID.
channel_ID = "2879119"

# The hostname of the ThingSpeak MQTT broker.
mqtt_host = "mqtt3.thingspeak.com"

# Your MQTT credentials for the device
mqtt_client_ID = "CDAxARguGDMsFxYCDDcmOjE"
mqtt_username  = "CDAxARguGDMsFxYCDDcmOjE"
mqtt_password  = "IUZXvw0Wpn17v6l+Rt9tPIMT"

t_transport = "websockets"
t_port = 80

# Create the topic string.
topic = "channels/" + channel_ID + "/publish"

# GitHub URL of the CSV file (raw URL)
CSV_URL = "https://raw.githubusercontent.com/le-pigeon/CEG5103/refs/heads/main/IoTSensorStream.csv"

# Reading the CSV file from GitHub
def read_csv_from_github(url):
    response = requests.get(url)
    csv_data = response.text
    df = pd.read_csv(io.StringIO(csv_data))
    return df

counter = 0
expected_count = 0
success_count = 0

# Automatically map CSV columns to ThingSpeak fields (skipping field1)
def send_data_to_thingspeak(df):
    global counter, expected_count, success_count
    expected_count = len(df)  # Total rows in CSV

    for index, row in df.iterrows():
        counter += 1
        sg_time = datetime.now(pytz.timezone('Asia/Singapore')).strftime("%Y-%m-%d %H:%M:%S")
        payload = ""  # Initialize the payload as empty

        # Map columns to ThingSpeak fields, starting from field1 (excluding field1)
        for i, column in enumerate(df.columns):
            if i > 0:  # Ensure we do not exceed ThingSpeak's max field count
                field_name = f"field{i}"  # Adjust field name based on ThingSpeak field indexing
                payload += f"{field_name}={row[column]}&"

        # Always append status=MQTTPUBLISH at the end
        payload += "status=MQTTPUBLISH"

        # Remove the last "&" from the payload if present (for cleaner output)
        payload = payload.rstrip("&")

        # attempt to publish this data to the topic.
        try:
          print(f"Upload {counter} at {sg_time}")
          print ("Writing Payload = ", payload," to host: ", mqtt_host, " clientID= ", mqtt_client_ID, " User ", mqtt_username, " PWD ", mqtt_password)
          publish.single(
              topic,
              payload,
              hostname=mqtt_host,
              transport=t_transport,
              port=t_port,
              client_id=mqtt_client_ID,
              auth={'username':mqtt_username,'password':mqtt_password}
          )
          success_count += 1  # Only increment when successfully published
          print(f"[{success_count}/{expected_count}] Data uploaded: {payload}")
        except Exception as e:
            print(f"Error while publishing data: {e}")
        time.sleep(5)


# Read and send data from CSV file
df = read_csv_from_github(CSV_URL)
send_data_to_thingspeak(df)

#time.sleep(7)  # Adjust the interval as needed

if success_count == expected_count:
        print("✅ 100% Data Successfully Uploaded!")
else:
        print(f"❌ Some data not uploaded. Success: {success_count}/{expected_count}")


Upload 1 at 2025-03-16 22:35:56
Writing Payload =  field1=147.2991029&field2=493.1729109&field3=131.4126515&field4=54.57653568&field5=0.0&status=MQTTPUBLISH  to host:  mqtt3.thingspeak.com  clientID=  CDAxARguGDMsFxYCDDcmOjE  User  CDAxARguGDMsFxYCDDcmOjE  PWD  IUZXvw0Wpn17v6l+Rt9tPIMT
[1/1873] Data uploaded: field1=147.2991029&field2=493.1729109&field3=131.4126515&field4=54.57653568&field5=0.0&status=MQTTPUBLISH
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Successfully connected to MQTT broker.
Upload 2 at 2025-03-16 22:36:02
Writing Payload =  field1=149.3906948&field2=425.9102458&field3=135.8148788&field4=46.57387499&field5=0.0&status=MQTTPUBLISH  to host:  mqtt3.thingspeak.com  clientID=  CDAxARguGDMsFxYCDDcmOjE  User  CDAxARguGDMsFxYCDDcmOjE  PWD  IUZXvw0Wp